In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.datasets import mnist
import matplotlib.pyplot as plt

In [5]:
(x_train, y_train), (_test, y_test) = mnist. load_data()
x_train = x_train.reshape (-1, 28, 28, 1) .astype ("float32") / 255.0
x_test = _test. reshape (-1, 28, 28, 1) .astype ("float32") / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [6]:
model = keras. Sequential([
    keras. Input ((28, 28, 1)),

#   data_augmentation,

    layers.Conv2D(4, 3, padding="same", activation="relu"),
    layers.Flatten(),
    layers. Dense (64, activation="relu"),
    layers. Dense (10),
],
                          name= 'model')

In [10]:
class CustomFit (keras.Model):

  def __init__ (self, model):
    super (CustomFit, self).__init__()
    self.model = model

  def train_step (self, data):
    x, y = data
    with tf. GradientTape() as tape: #in this we do forward prop and loss calc, so that it can keep track and do backprop
      y_pred = self.model(x, training=True)
      loss=self.compiled_loss(y, y_pred) #this compiled loss is using sparsecatentrop. from compile

    trainable_vars=self.trainable_variables #these are from parent class so dont bother
    gradients=tape.gradient(loss, trainable_vars) #calculating loss with respect to trainable variables

    #now to gradient decent step for optimizing
    self.optimizer.apply_gradients(zip(gradients, trainable_vars))
    self.compiled_metrics.update_state(y, y_pred) #this is for accuracy

    return {m.name: m.result() for m in self.metrics}


In [12]:
training= CustomFit(model)

training.compile(optimizer=keras.optimizers.Adam(),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=["accuracy"],
)

training.fit(x_train, y_train,
          epochs=5,
          verbose=2)

Epoch 1/5
1875/1875 - 17s - loss: 0.0278 - accuracy: 0.9909 - 17s/epoch - 9ms/step
Epoch 2/5
1875/1875 - 16s - loss: 0.0197 - accuracy: 0.9936 - 16s/epoch - 8ms/step
Epoch 3/5
1875/1875 - 19s - loss: 0.0159 - accuracy: 0.9948 - 19s/epoch - 10ms/step
Epoch 4/5
1875/1875 - 16s - loss: 0.0128 - accuracy: 0.9959 - 16s/epoch - 9ms/step
Epoch 5/5
1875/1875 - 22s - loss: 0.0112 - accuracy: 0.9963 - 22s/epoch - 12ms/step


In [22]:
#we can do our own compile also

class CustomFit (keras.Model):

  def __init__ (self, model):
    super (CustomFit, self).__init__()
    self.model = model

  def compile(self, loss, optimizer):
    super(CustomFit, self).compile()
    self.optimizer=optimizer
    self.loss=loss

  def train_step (self, data):
    x, y = data
    with tf. GradientTape() as tape:
      y_pred = self.model(x, training=True)
      loss=self.loss(y, y_pred)

    trainable_vars=self.trainable_variables
    gradients=tape.gradient(loss, trainable_vars)


    self.optimizer.apply_gradients(zip(gradients, trainable_vars))

    acc_metric.update_state(y, y_pred)#this is for accuracy

    return {'loss': loss, 'accuracy': acc_metric.result()}

  def test_step(self, data):
    x, y = data
    y_pred = self .model(x, training=False)
    loss = self.loss (y, y_pred)
    acc_metric.update_state(y, y_pred)

    return {'Loss': loss, "accuracy": acc_metric.result()}


In [23]:
acc_metric=keras.metrics.SparseCategoricalAccuracy(name='accuracy')

In [24]:
training= CustomFit(model)

training.compile(optimizer=keras.optimizers.Adam(),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

training.fit(x_train, y_train,
          epochs=5,
          verbose=2)

Epoch 1/5
1875/1875 - 21s - loss: 1.1581e-05 - accuracy: 0.9982 - 21s/epoch - 11ms/step
Epoch 2/5
1875/1875 - 15s - loss: 6.9759e-05 - accuracy: 0.9986 - 15s/epoch - 8ms/step
Epoch 3/5
1875/1875 - 16s - loss: 4.1396e-05 - accuracy: 0.9986 - 16s/epoch - 8ms/step
Epoch 4/5
1875/1875 - 16s - loss: 8.4388e-04 - accuracy: 0.9986 - 16s/epoch - 9ms/step
Epoch 5/5
1875/1875 - 16s - loss: 1.2704e-05 - accuracy: 0.9986 - 16s/epoch - 8ms/step


In [25]:
training.evaluate(x_test, y_test)

313/313 [==============================] - 1s 4ms/step - Loss: 0.1226 - accuracy: 0.9983


[3.725289943190546e-08, 0.9981064796447754]